# CA2_Submission_LineChart(Bokeh) using SFA dataset


In [1]:
!pip install pandas bokeh

In [1]:
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Select, CustomJS, Legend, LegendItem
from bokeh.layouts import column
from bokeh.io import curdoc

In [2]:
#load the data and check
df = pd.read_csv("ValueofLocalFoodProductioninSingapore.csv")
df.head()

,year,vegetables,seafood,hen_shell_eggs,local_landings
0,2012,28.6,16.4,48.4,14.8
1,2013,28.9,34.6,59.0,14.3
2,2014,29.6,31.4,63.2,12.6
3,2015,30.6,39.4,56.7,13.1
4,2016,34.0,36.8,55.8,12.7


In [5]:
### Text-based Analysis Summary using Pandas ###

# Display shape of the dataset
print("<< This is the shape of the dataset >>")
print(df.shape)
print()

# Display index range
print("<< This is the index of the dataset >>")
print(df.index)
print()

# Display all columns in the dataset
print("<< These are the columns in the dataset >>")
print(df.columns)
print()

# Display summary of the dataset
print("<< A summary of this dataset is shown below >>")
print(df.info())
print()

# Count total number of non-NA (non-null) values in the dataset
print("<< The total number of non-NA values in this dataset >>")
print(df.count())
print()

# Display statistical summary of the quantitative variables columns (**not required for this task**)
print("<< A descriptive statistical summary of the quantitative data in this dataset >>")
print(df.describe())

<< This is the shape of the dataset >>
(9, 5)

<< This is the index of the dataset >>
RangeIndex(start=0, stop=9, step=1)

<< These are the columns in the dataset >>
Index(['year', 'vegetables', 'seafood', 'hen_shell_eggs', 'local_landings'], dtype='object')

<< A summary of this dataset is shown below >>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            9 non-null      int64  
 1   vegetables      9 non-null      float64
 2   seafood         9 non-null      float64
 3   hen_shell_eggs  9 non-null      float64
 4   local_landings  9 non-null      float64
dtypes: float64(4), int64(1)
memory usage: 492.0 bytes
None

<< The total number of non-NA values in this dataset >>
year              9
vegetables        9
seafood           9
hen_shell_eggs    9
local_landings    9
dtype: int64

<< A descriptive statistical summary of the qua

In [3]:
# Convert year to string for categorical x-axis
df['year'] = df['year'].astype(str)
# Create a ColumnDataSource (Bokeh data structure to be used to create plot)
source = ColumnDataSource(df)

# Output to a static HTML file for interactive visualization
output_file("interactive_line_chart.html")

# Create figure
p = figure(x_range=df['year'].tolist(), width=800, height=500, title="Value of Local Food Production in Singapore",
           toolbar_location=None, tools="hover")

# Add line without legend_label
line = p.line(x='year', y='vegetables', source=source, color="green", line_width=2)
circle = p.circle(x='year', y='vegetables', source=source, color="green", size=8)

# Configure axes
p.xaxis.axis_label = "Year"
p.yaxis.axis_label = "Values($M)"
p.xgrid.grid_line_color = None

# Add hover tooltips
hover = p.select_one(HoverTool)
hover.tooltips = [("Year", "@year"), ("Value($M)", "@vegetables")]

# Create initial legend
legend_item = LegendItem(label="Vegetables", renderers=[line])
legend = Legend(items=[legend_item], location="top_left", orientation="horizontal")
p.add_layout(legend)

# Create dropdown menu
select = Select(title="Category", value="vegetables", options=["vegetables", "seafood", "hen_shell_eggs", "local_landings"])

# CustomJS callback - JavaScript code to update the plot based on the dropdown selection
callback = CustomJS(args=dict(line=line, circle=circle, hover=hover, legend_item=legend_item), code="""
    var category = cb_obj.value;
    line.glyph.y.field = category;
    circle.glyph.y.field = category;
    line.data_source.change.emit();
    circle.data_source.change.emit();
    hover.tooltips = [["Year", "@year"], ["Value($M)", "@" + category]];
    
    // Update legend
    legend_item.label = category.charAt(0).toUpperCase() + category.slice(1);
""")

# Link dropdown to callback to update the chart
select.js_on_change("value", callback)

# Layout (dropdown menu and plot in vertical layout) and show the plot
layout = column(select, p)
show(layout)
